10000 сообщений нормировались за 20 минут

In [1]:
import scaning_message
import time

import gensim
from gensim import corpora
from pprint import pprint
from gensim.utils import simple_preprocess


from gensim.models.word2vec import Word2Vec
from multiprocessing import cpu_count


from gensim.models import Word2Vec, KeyedVectors  

import numpy as np

In [2]:
#сохраняет модель на диске
def save_model(model,name='model'):
    model.wv.save_word2vec_format(f'{name}.bin', binary=True)

#загружает модель с диска
def download_model(name='model'):
    model = KeyedVectors.load_word2vec_format('{name}.bin', binary=True)
    

def create_model(data):
    start_time = time.time()
    model = Word2Vec(data, min_count = 0,size=50, workers=cpu_count())
    print(f"на создание модели потребовалось {time.time() - start_time} секунд")
    return (model)


def synonyms_word(word,model):
    synonyms=model.wv.most_similar(positive=[f"{word}"])
    return (synonyms)    

In [3]:
#нормирует документ сохраняя результат в новом  
def raw_to_normal(start_name,finish_name):
    start_time = time.time()
    file_start=open(f'{start_name}.txt','r')
    total_lines=0
    
    for line in file_start:
        total_lines+=1
    file_start.close()
    #total_lines=1000#для теста
    count_lines=0
    
    file_start=open(f'{start_name}.txt','r')
    file_finish=open(f'{finish_name}.txt','w')
    for line in file_start:
        file_finish.write(scaning_message.normalize_txt(line)+'\n')
        count_lines+=1
        
        scaning_message.progress(count_lines,total_lines,"нормировано")
        
        if count_lines== total_lines:
            break
    file_start.close()
    file_finish.close()
    print(f"нормировка заняла {time.time() - start_time} секунд")

def get_normalized_data(name_file):
    data=[]
    file=open(f'{name_file}.txt','r')
    for line in file:
        data.append(line)
    file.close()
    for i in range(len(data)):
        data[i]=data[i].split(' ')
        data[i][-1]=data[i][-1][:len(data[i][-1])-1]
        for j in range (len(data[i])):
            if 'id' in data[i][j]:
                data[i][j]=''
        if ' ' in data[i]:
            data[i].remove(' ')
        elif '...' in data[i]:
            data[i].remove('...')
        elif '' in data[i]:
            data[i].remove('')
        elif ',' in data[i]:
            data[i].remove(',')
        elif ':' in data[i]:
            data[i].remove(':')
        elif '-' in data[i]:
            data[i].remove('-')
        elif ']' in data[i]:
            data[i].remove(']')
        elif '[' in data[i]:
            data[i].remove('[')
        
    return (data)

def get_vector(word,model):
    return model[word]
    

In [4]:
raw_to_normal('more_coments','norm')

нормировка заняла 19065.07349538803 секунд 100.0% ...нормировано


In [4]:
data=get_normalized_data('norm1')

In [5]:
model=create_model(data)

на создание модели потребовалось 56.51369905471802 секунд


In [6]:
synonyms_word('кот',model)

[('кошка', 0.8982638716697693),
 ('пёс', 0.8309276103973389),
 ('котёнок', 0.8248428702354431),
 ('собачка', 0.7985167503356934),
 ('собака', 0.7932366728782654),
 ('хозяйка', 0.7278547883033752),
 ('хася', 0.7276586890220642),
 ('котик', 0.718923032283783),
 ('съедение', 0.6872754693031311),
 ('ребёночек', 0.6872730255126953)]

In [7]:
print(get_vector('кот',model))

[-0.08659966  1.9326174  -0.38252205  0.9002273   0.32290027 -0.36020696
 -1.222435    1.4398873  -1.9707593  -1.2629893   1.4483807  -1.0553977
  0.8041711   1.4306101   1.1315747  -1.6722149   2.204809    0.68184084
  0.625223    0.5985732  -1.1109047  -0.5927945   0.35397062  0.38531822
  2.9079354  -0.85332316 -3.361672    3.542375   -0.0654007  -0.02130559
  1.0551704   0.6429949  -2.0501316   0.03897918 -0.45482633  0.62069815
 -0.55301285 -0.46896544  1.5235717  -1.3742055   0.7973823  -0.82711196
 -1.5434997   0.65282184 -0.6307232  -0.71468544  1.0442744  -0.28842247
  1.2780718  -1.6499374 ]


<ipython-input-3-bb3865d35514>:59: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  return model[word]


# #создание базовой обучающей выборки

In [7]:
file = open('bad_words.txt','r')
x=[]
for line in file:
    x.append(line)
bad_words=x[0].split(', ')
file.close()

In [8]:
##Создание файлов с плохими словами и без

In [11]:
count_bad=0
all_phrases=open('norm1.txt','r')
bad_phrases1= open('bad_phrases1.txt','w')
good_phrases1= open('good_phrases1.txt','w')
text=[]
for line in all_phrases:
    text.append(line)
for i in range(len(data)):
    flag=0
    for word in data[i]:
        if word in bad_words:
            flag=1
            count_bad+=1
            bad_phrases1.write(text[i])
            break
    if flag==0:
        
        good_phrases1.write(text[i])
bad_phrases1.close()
good_phrases1.close()
            
print(count_bad)

NameError: name 'text' is not defined

In [8]:
#Разметка
#Первый нейрон отчечает за нормальные аудиозаписи, второй за оскорбительные
count_bad=0
y_data=[]

for i in range(len(data)):
    flag=0
    for word in data[i]:
        if word in bad_words:
            flag=1
            count_bad+=1
            y_data.append(np.array([0,1]))
            break
    if flag==0:
        y_data.append(np.array([1,0]))
    
y_data=np.array(y_data)

In [ ]:
import copy

In [9]:
max_words=100
z= np.zeros(50)
x_data=data[:50000]
for n in range(len(x_data)):
    for n_word in range(len(x_data[n])):
        try:
            x_data[n][n_word]=get_vector(x_data[n][n_word],model)
        except:
            x_data[n][n_word]=z

        
        
    if len(x_data[n])>max_words:
        x_data[n]=x_data[n][:max_words]
    else:
        while len(x_data[n])<max_words:
            x_data[n].insert(0,z)
        
    
    x_data[n]=np.array(x_data[n])
    
x_data=np.array(x_data)

        

<ipython-input-3-bb3865d35514>:59: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  return model[word]


In [20]:
len(x_data)

50000

In [19]:
count=0
for i in range(len(x_data)):
        if y_data[i][1]==1:
            count+=1
print(count)

3541


In [13]:
X=[]
Y=[]

X.append(x_data[0])
Y.append(y_data[0])
for i in range (len(x_data)-1):
    if y_data[i][0]!=Y[len(Y)-1][0]:
        X.append(x_data[i])
        Y.append(y_data[i])

X=np.array(X)
Y=np.array(Y)

In [15]:
len(Y)

6187

In [21]:
from tensorflow.keras.layers import Dense, SimpleRNN, Input, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.utils import to_categorical

In [22]:
network_model = Sequential()
network_model.add(Input((100, 50)))
network_model.add(Dense(units=50, activation='linear'))
network_model.add(LSTM(50, return_sequences=True))
network_model.add(LSTM(50))
network_model.add(Dense(units=10, activation='linear'))
network_model.add(Dense(2, activation='softmax'))
network_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100, 50)           2550      
_________________________________________________________________
lstm (LSTM)                  (None, 100, 50)           20200     
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                510       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 22        
Total params: 43,482
Trainable params: 43,482
Non-trainable params: 0
_________________________________________________________________


In [23]:
network_model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [24]:
history = network_model.fit(X, Y, batch_size=32, epochs=30, validation_split=0.1)

Epoch 1/30
174/174 [==============================] - 14s 53ms/step - loss: 0.5980 - accuracy: 0.6687 - val_loss: 0.4166 - val_accuracy: 0.7964
Epoch 2/30
174/174 [==============================] - 8s 45ms/step - loss: 0.3984 - accuracy: 0.8291 - val_loss: 0.3968 - val_accuracy: 0.8401
Epoch 3/30
174/174 [==============================] - 8s 45ms/step - loss: 0.3178 - accuracy: 0.8629 - val_loss: 0.3535 - val_accuracy: 0.8675
Epoch 4/30
174/174 [==============================] - 8s 45ms/step - loss: 0.2783 - accuracy: 0.8842 - val_loss: 0.3203 - val_accuracy: 0.8627
Epoch 5/30
174/174 [==============================] - 8s 46ms/step - loss: 0.2379 - accuracy: 0.8996 - val_loss: 0.2977 - val_accuracy: 0.8918
Epoch 6/30
174/174 [==============================] - 8s 45ms/step - loss: 0.1848 - accuracy: 0.9233 - val_loss: 0.2837 - val_accuracy: 0.8821
Epoch 7/30
174/174 [==============================] - 8s 45ms/step - loss: 0.1946 - accuracy: 0.9214 - val_loss: 0.3154 - val_accuracy: 0.867

In [ ]:
network_model